In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [24]:
param = {}

Retreive instagram credentials from files in this directory

In [6]:
CLIENT_ID = open('client_id.txt').read()
ACCESS_TOKEN = open('access_token.txt').read()

Scrape the instagram handles from Time's Instagram Followers To Follow in All 50 States 

In [7]:
url = 'http://time.com/instagram50/'
res = requests.get(url)
soup = BeautifulSoup(res.text)

In [8]:
user_names = []
for text in soup.find_all('span', "username"):
    user_names.append(text.string[1:])

Add other photographer accounts

In [9]:
user_names.extend([])

Get data available using instagram handles/usernames

In [10]:
user_ids = []
follower_counts = []
for user_name in user_names:
    url = 'https://api.instagram.com/v1/users/search?q={}&client_id={}'.format(user_name, CLIENT_ID)
    res = requests.get(url,params=param)
    res = res.json()
    user_id = int(res['data'][0]['id'])
    user_ids.append(user_id)
    
    url = 'https://api.instagram.com/v1/users/{}/?access_token={}'.format(user_id, ACCESS_TOKEN)
    res = requests.get(url,params=param)
    res = res.json()
    follower_count = res['data']['counts']['followed_by']
    follower_counts.append(follower_count)

In [11]:
id_follower_counts = dict(zip(user_ids, follower_counts))

Clean up and put in pandas dataframe

In [12]:
photo_details = []
for user_id in user_ids:
    url = 'https://api.instagram.com/v1/users/{}/media/recent/?access_token={}'.format(user_id, ACCESS_TOKEN)
    res = requests.get(url,params=param)
    res = res.json()
    
    for i in range(0, 20):
        post = res['data'][i]
        like_count = post['likes']['count']
        image_url = post['images']['standard_resolution']['url']
        created_at = post['created_time']
        tag_count = len(post['users_in_photo'])
        post_type = post['type']
        location = post['location']
        if location:
            if 'name' in location.keys():
                location = location['name']
        photo_details.append({'user_id': user_id, 
                              'like_count': like_count, 
                              'image_url': image_url, 
                              'created_time': created_at,
                              'tag_count': tag_count,
                              'post_type': post_type,
                              'location': location
                             })

In [13]:
photo_details = pd.DataFrame(photo_details)

Clean and add columns

In [14]:
photo_details['created_time'] = pd.to_datetime(photo_details.created_time.astype(int), unit='s')

In [17]:
def scale_like_counts_relative_to_account(row):
    photo_like_count = row.like_count
    account_photo_likes = photo_details[photo_details.user_id == row.user_id].like_count
    photo_like_count *= 1.0 / (account_photo_likes.max() - account_photo_likes.min())
    photo_like_count -= account_photo_likes.min() * (1.0 / (account_photo_likes.max() - account_photo_likes.min()))
    
    return photo_like_count

photo_details['scaled_likes'] = photo_details.apply(scale_like_counts_relative_to_account, axis=1)

In [18]:
def likes_per_follower(row):
    return float(row.like_count)/id_follower_counts[row.user_id]

photo_details['likes_per_follower'] = photo_details.apply(likes_per_follower, axis=1)

In [19]:
def location_exists(row):
    if row.location:
        return True
    return False

photo_details['location_exists'] = photo_details.apply(location_exists, axis = 1)

In [20]:
def tags_exist(row):
    if row.tag_count > 0:
        return True
    return False

photo_details['tags_exist'] = photo_details.apply(tags_exist, axis = 1)

In [21]:
def hour_of_day(row):
    return row.created_time.hour

def day_of_week(row):
    return row.created_time.weekday()

photo_details['hour_of_day'] = photo_details.apply(hour_of_day, axis=1)
photo_details['day_of_week'] = photo_details.apply(day_of_week, axis=1)